##1-Use python requests package to get the file from here

In [10]:
import requests
import pandas as pd
import numpy as np
import re


url = "https://raw.githubusercontent.com/victorbrub/data-engineering-class/refs/heads/main/pre-post_processing/exercise.csv"
filename = "exercise_raw.csv"

response = requests.get(url)

# We saved the downloaded content to a local file
with open(filename, "wb") as f:
    f.write(response.content)

print("File downloaded successfully as:", filename)


File downloaded successfully as: exercise_raw.csv


##2-Check the file data to fast check. Read it with Pandas.

  ### a) How many rows do we have?

In [11]:
df = pd.read_csv((filename), on_bad_lines='skip')
print("First rows of the dataset:")
display(df.head())

print("\nDataset information:")
df.info()

print(f"\nNumber of rows: {len(df)}")
print(f"Number of columns: {df.shape[1]}")

First rows of the dataset:


,OrderID,CustomerName,Email,Phone,Country,OrderDate,Quantity,Price,CustomerAge,OrderStatus
0,1001,John Doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
1,1002,jane smith,JANE@GMAIL.COM,555-2345,united states,2023-01-16,2,49.99,35,Completed
2,1003,john doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
3,1004,Bob Johnson,bob.j@email.com,555-3456,US,2023-01-17,-1,99.99,45,Pending
4,1006,Charlie Davis,NaN,555-5678,UK,2023-02-19,3,79.99,unknown,Completed



Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OrderID       189 non-null    int64  
 1   CustomerName  189 non-null    object 
 2   Email         165 non-null    object 
 3   Phone         184 non-null    object 
 4   Country       189 non-null    object 
 5   OrderDate     189 non-null    object 
 6   Quantity      189 non-null    int64  
 7   Price         168 non-null    float64
 8   CustomerAge   187 non-null    object 
 9   OrderStatus   189 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 14.9+ KB

Number of rows: 189
Number of columns: 10


There are 189 rows

### b) Is there any sensible information?

The dataset analysis identified several columns containing personally identifiable information (PII). Among these, the Email and Phone fields are considered sensitive data, as they allow direct contact with the individual, and their exposure could pose risks such as fraud, identity theft, or unsolicited communications. Other columns, such as CustomerName, CustomerAge, and Country, also form part of the PII because they help identify a person, although they are not considered sensitive information in and of themselves. Finally, attributes such as OrderID, OrderDate, Quantity, Price, and OrderStatus do not contain personal information; they only describe transaction characteristics. This distinction is important for determining the level of protection and handling required for each type of data within the cleaning and analysis process.

### c) What kind of problems can we have regarding the nature of this data?

In [12]:
display(df)

,OrderID,CustomerName,Email,Phone,Country,OrderDate,Quantity,Price,CustomerAge,OrderStatus
0,1001,John Doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
1,1002,jane smith,JANE@GMAIL.COM,555-2345,united states,2023-01-16,2,49.99,35,Completed
2,1003,john doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28,Completed
3,1004,Bob Johnson,bob.j@email.com,555-3456,US,2023-01-17,-1,99.99,45,Pending
4,1006,Charlie Davis,NaN,555-5678,UK,2023-02-19,3,79.99,unknown,Completed
...,...,...,...,...,...,...,...,...,...,...
184,1186,Wren Park,wren@email.com,invalid_phone,Canada,2023-08-17,3,34.99,999,Completed
185,1187,Zeke Lynn,zeke@email.com,555-5757,USA,2023/08/18,5,54.99,unknown,Processing
186,1188,Aria West,aria.west@email,555 6868,United Kingdom,2023-08-19,-3,69.99,-28,Pending
187,1189,Bryn Cole,BRYN@EMAIL.COM,555-7979,usa,08/20/2023,8,NaN,43,Shipped


The dataset presents several problems related to the nature and quality of the data it contains. First, there are clear formatting inconsistencies, such as variations in the use of uppercase and lowercase letters in names and email addresses, and multiple different ways of writing the country (USA, US, usa, united states, United Kingdom). There are also invalid values, such as negative numbers in Quantity and CustomerAge, out-of-range ages (such as 999 or -28), and dates with incorrect formats (08/20/2023). Furthermore, there are records with invalid or malformed data, such as phone numbers containing text (invalid_phone), and null values, such as in Email (NaN). There are also duplicates, with the same customers having the same name, email, and phone number (John Doe). Finally, the mixture of sensitive personal data with errors and inconsistent formats increases the risk of mishandling the data and highlights the need to apply data cleansing processes before analysis.

##3-Clean it.

### a) Define the rules we need to clean the data.

**Cleaning rules based on the anomalies detected in the dataset:**


1. Text Standardization:
     *   Standardize names to the format First Name Last Name (John Doe).
     *   Convert all email addresses to lowercase.
     *   Remove extra spaces from all text columns.
     *   Unify the order status format ("Completed", "Pending", "Shipped").
<br>

2.   Standardization of countries:
     *   Unify variants into a single standard format:
         *   "us", "usa", "united states" : "USA"
         *   "uk", "united kingdom" : "UK"
         *   "canada" : "Canada"
<br>

3.   Email validation:
     *   Mark invalid emails or those identified as "invalid_email" as missing values (NaN).
     *   Only accept email addresses with a valid format.

<br>

4.   Phone number cleanup:
     *   Replace "invalid_phone" with NaN.
     *   Remove invalid characters, leaving only digits or hyphens.
     *   If the phone number is still invalid after cleaning, replace it with NaN.

<br>

5.   Customer age validation:
     *   Convert the age column to numeric values.
     *   Replace non-numeric values such as "unknown" with NaN.
     *   Set as invalid any ages outside the reasonable range (less than 0 or greater than 120).

<br>

6.   Quantity and price validation:
     *   Convert Quantity and Price to numeric values.
     *   Only positive values are considered valid.
     *   Rows with negative, null, or non-numeric quantities or prices will be deleted.

<br>

7.   Date correction:
     *   Convert the OrderDate column to datetime format.
     *   Dates with incorrect formats are marked as NaT.

<br>

8.   Detección y eliminación de duplicados:
     *   Identify duplicate records where all values are the same except for OrderID.
     *   Keep only the first occurrence of each duplicate.

<br>

These rules allow for a coherent cleaning process, correcting invalid values, standardizing formats, and ensuring that the dataset is ready for subsequent analysis and quality testing.

### b) Implement a Python Script that cleans the data and analyzes the clean process.


In [13]:
# We make a copy so we don't have to touch the original.
df_cop = df.copy()

# 1. Clean up whitespace in all text columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip()

# 2.Standardize names, emails, countries, and order status

# CustomerName in title format (John Doe)
df["CustomerName"] = df["CustomerName"].str.lower().str.title()

# Email in lowercase, "invalid_email" is considered a missing value
df["Email"] = df["Email"].str.lower()
df["Email"] = df["Email"].replace("invalid_email", np.nan)

# Country in the same format: we map to standard values
country_map = {
    "usa": "USA",
    "us": "USA",
    "united states": "USA",
    "united kingdom": "UK",
    "uk": "UK",
    "canada": "Canada"
}
df["Country"] = (
    df["Country"]
    .str.lower()
    .map(country_map)
)

# OrderStatus in title format (Completed, Pending…)
df["OrderStatus"] = df["OrderStatus"].str.lower().str.title()

# 3.Clean phone: "invalid_phone" is marked as NaN and we remove any characters that are not digits or hyphens
df["Phone"] = df["Phone"].replace("invalid_phone", np.nan)

def clean_phone(x):
    if pd.isna(x):
        return np.nan
    # Dejamos solo dígitos y guion
    x = re.sub(r"[^0-9-]", "", x)
    return x if x else np.nan

df["Phone"] = df["Phone"].apply(clean_phone)

# 4. Convert types and validate numeric ranges: OrderDate to datetime, Quantity and Price to numeric and positive, CustomerAge to numeric within a reasonable range (0–120), and
#"unknown" or impossible values ​-> NaN

# Date
df["OrderDate"] = pd.to_datetime(df["OrderDate"], errors="coerce")

# Quantity: only values ​​> 0
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
mask_qty = df["Quantity"] > 0

# Price: only values ​​> 0
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
mask_price = df["Price"] > 0

# Customer age
df["CustomerAge"] = pd.to_numeric(df["CustomerAge"], errors="coerce")
# Marcamos como NaN edades fuera de rango razonable
df.loc[(df["CustomerAge"] < 0) | (df["CustomerAge"] > 120), "CustomerAge"] = np.nan

# 5. Remove rows with clearly invalid data in the transaction (invalid quantity or price)
df = df[mask_qty & mask_price].copy()

# 6. Remove duplicate orders: If all columns except OrderID are the same, we keep
#only one record (the first record) and discard the rest

cols_for_dupes = [c for c in df.columns if c != "OrderID"]
df = df.drop_duplicates(subset=cols_for_dupes, keep="first")

print("Filas originales:", len(df_cop))
print("Filas después de la limpieza:", len(df))
print("Archivo limpio guardado como 'exercise_clean.csv'")

Filas originales: 189
Filas después de la limpieza: 119
Archivo limpio guardado como 'exercise_clean.csv'


In [14]:
display(df)

,OrderID,CustomerName,Email,Phone,Country,OrderDate,Quantity,Price,CustomerAge,OrderStatus
0,1001,John Doe,john@email.com,555-1234,USA,2023-01-15,5,19.99,28.0,Completed
1,1002,Jane Smith,jane@gmail.com,555-2345,USA,2023-01-16,2,49.99,35.0,Completed
4,1006,Charlie Davis,nan,555-5678,UK,2023-02-19,3,79.99,NaN,Completed
5,1007,David Miller,david@email.com,555-6789,NaN,2023-02-20,1,199.99,NaN,Processing
6,1008,Eve Wilson,eve.wilson@yahoo.com,5557890,USA,NaT,7,14.99,29.0,Completed
...,...,...,...,...,...,...,...,...,...,...
181,1183,Skye Frost,nan,555-2424,USA,NaT,4,44.99,39.0,Pending
182,1184,Trey Banks,trey.banks@email.com,5553535,USA,2023-08-15,7,19.99,NaN,Shipped
184,1186,Wren Park,wren@email.com,NaN,Canada,2023-08-17,3,34.99,NaN,Completed
185,1187,Zeke Lynn,zeke@email.com,555-5757,USA,NaT,5,54.99,NaN,Processing


##4-Output should be a new file with cleaned data

In [15]:
df.to_csv("exercise_clean.csv", index=False)

##5-Create a test on the raw data and on the cleaned data, for each one of the dimensions. The output should be a percentage: rows that meet the test requirement / total rows. Compare the results of each file.

In [16]:
df_raw = pd.read_csv("exercise_raw.csv", engine="python", on_bad_lines="skip")
df_clean = pd.read_csv("exercise_clean.csv")

email_pattern = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"
valid_countries = ["USA", "UK", "Canada"]

def run_tests(df):
    total = len(df)

    tests = {}

    tests["valid_name"] = df["CustomerName"].notna()
    tests["valid_email"] = df["Email"].astype(str).str.match(email_pattern, na=False)
    tests["valid_phone"] = df["Phone"].astype(str).str.contains(r"^\d{3,}-\d{3,}$", na=False)
    tests["valid_country"] = df["Country"].isin(valid_countries)
    tests["valid_quantity"] = pd.to_numeric(df["Quantity"], errors="coerce") > 0
    tests["valid_price"] = pd.to_numeric(df["Price"], errors="coerce") > 0

    age = pd.to_numeric(df["CustomerAge"], errors="coerce")
    tests["valid_age"] = (age >= 0) & (age <= 120)

    tests["valid_date"] = pd.to_datetime(df["OrderDate"], errors="coerce").notna()

    # Convert to percentages
    results = {k: round(v.mean() * 100, 2) for k, v in tests.items()}
    return results




In [17]:
raw_results = run_tests(df_raw)
clean_results = run_tests(df_clean)

In [18]:
comparison = pd.DataFrame({
    "Raw Data (%)": raw_results,
    "Clean Data (%)": clean_results
})

comparison

,Raw Data (%),Clean Data (%)
valid_name,100.00,100.00
valid_email,62.96,73.95
valid_phone,68.25,73.11
valid_country,50.26,98.32
valid_quantity,76.72,100.00
valid_price,88.89,100.00
valid_age,48.15,47.90
valid_date,62.96,67.23


After applying the tests to both the original and cleaned datasets, a general improvement is observed in most dimensions. In the cleaned dataset, validity percentages increase for email, phone number, country, quantity, and price, thanks to format normalization and the elimination of negative or invalid values. The most significant improvement occurs in Country, which jumps from 50% to over 98% due to the standardization of all variants.

The Quantity and Price variables reach 100% validity after cleaning, as rows with invalid values ​​were removed. In contrast, CustomerAge barely changes because the process only marks impossible ages as invalid, but does not correct those values. Date validity also improves slightly by converting them to the correct format.

Overall, the results show that cleaning has increased the consistency and quality of the dataset, reducing errors and leaving the data in better condition for subsequent analysis.